In [7]:
import pandas as pd

# 加载数据集
df = pd.read_csv('dft-road-casualty-statistics-collision-last-5-years.csv')

# 显示数据框信息
print("数据集形状:", df.shape)
print("\n前几行数据:")
print(df.head())
print("\n数据类型:")
print(df.dtypes)
print("\n数据摘要:")
print(df.info())

数据集形状: (503475, 44)

前几行数据:
  collision_index  collision_year collision_ref_no  location_easting_osgr  \
0   2021170H10421            2021        170H10421               447098.0   
1   2021170H11231            2021        170H11231               450486.0   
2   2020170M11750            2020        170M11750               449694.0   
3   2021170M31761            2021        170M31761               449744.0   
4   2021170S10441            2021        170S10441               445971.0   

   location_northing_osgr  longitude   latitude  police_force  \
0                532997.0  -1.270905  54.689833            17   
1                533118.0  -1.218333  54.690592            17   
2                519733.0  -1.232884  54.570397            17   
3                514217.0  -1.233040  54.520825            17   
4                520834.0  -1.290292  54.580641            17   

   collision_severity  number_of_vehicles  ...  carriageway_hazards_historic  \
0                   3                 

In [11]:
# 按LSOA和月份统计事故数，创建透视表格式
# 首先确保date列是datetime类型
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

# 提取年月
df['year_month'] = df['date'].dt.to_period('M')

# 按lsoa_of_accident_location和year_month分组，统计事故数
lsoa_monthly_counts = df.groupby(['lsoa_of_accident_location', 'year_month']).size().reset_index(name='accident_count')

# 创建透视表：lsoa作为行，年月作为列
lsoa_monthly_pivot = lsoa_monthly_counts.pivot(index='lsoa_of_accident_location', 
                                                columns='year_month', 
                                                values='accident_count').fillna(0)

print("以LSOA为行、月份为列的透视表:")
print(lsoa_monthly_pivot.head())
print(f"\n形状: {lsoa_monthly_pivot.shape}")
print(f"行数（LSOA数）: {lsoa_monthly_pivot.shape[0]}")
print(f"列数（月数）: {lsoa_monthly_pivot.shape[1]}")
print(f"\n月份范围: {lsoa_monthly_pivot.columns.min()} 到 {lsoa_monthly_pivot.columns.max()}")

# 也可以查看统计信息
print(f"\n每个LSOA的总事故数统计:")
print(lsoa_monthly_pivot.sum(axis=1).describe())

# 保存到本地CSV文件
output_file = 'lsoa_monthly_accidents.csv'
lsoa_monthly_pivot.to_csv(output_file)
print(f"\n✓ 已保存到: {output_file}")

以LSOA为行、月份为列的透视表:
year_month                 2020-01  2020-02  2020-03  2020-04  2020-05  \
lsoa_of_accident_location                                                
-1                           412.0    402.0    289.0    155.0    218.0   
E01000001                      0.0      1.0      0.0      0.0      0.0   
E01000002                      0.0      1.0      0.0      0.0      0.0   
E01000003                      0.0      0.0      0.0      0.0      0.0   
E01000005                      1.0      0.0      1.0      0.0      0.0   

year_month                 2020-06  2020-07  2020-08  2020-09  2020-10  ...  \
lsoa_of_accident_location                                               ...   
-1                           278.0    326.0    432.0    357.0    353.0  ...   
E01000001                      2.0      0.0      2.0      0.0      2.0  ...   
E01000002                      0.0      0.0      0.0      0.0      0.0  ...   
E01000003                      0.0      0.0      0.0      0.0      0